In [1]:
from configparser import ConfigParser
parser = ConfigParser()
_ = parser.read('notebook.cfg')
eventHubConnectString = parser.get('my_Notebook','eventHubConnectString')

In [2]:
 # pip install azure-eventhub

In [3]:
eventhub_name = 'speedtest'
sleepMinutes = 10

In [4]:
# pip install speedtest-cli

In [5]:
import os
import speedtest 

In [6]:
import socket
import requests
import json

In [7]:
def test():
    s = speedtest.Speedtest()
    ip = socket.gethostbyname(socket.gethostname())
    ipUrl = "https://api.ipify.org/?format=json"
    response= json.loads(requests.get(ipUrl).text)
    print(response)
    print(response["ip"])
    externalIp = response["ip"]
    s.get_servers()
    s.get_best_server()
    s.download()
    s.upload()
    res = s.results.dict()
    return res["download"], res["upload"], res["ping"],externalIp

In [8]:
import datetime
import time

In [9]:
from azure.eventhub import EventHubProducerClient, EventData
def SendToEventHub(data):
    try:
        print("enter SendToEventHub", data)
        client = EventHubProducerClient.from_connection_string(eventHubConnectString, eventhub_name=eventhub_name)
        event_data_batch = client.create_batch()
        event_data_batch.add(EventData(data))
        with client:
            client.send_batch(event_data_batch)
        print("Sent data")
    except Exception as e:
        print("Unable to send data to event hub",e)

In [10]:
import time
import calendar
def GetTimestamp():
    gmTime= time.gmtime()    
    now = calendar.timegm( gmTime)
    print("GetTimestamp",now,gmTime)
    return now

In [ ]:
import time
print("ready to start work")
linesToWrite = []
while (True):  
    try:
        now = GetTimestamp()
        d, u, p,ip = test()
        print(d,u,p,ip)
    except  Exception as ex:
        print("exception",ex)
        d = -1
        u = -1
        p = -1
        ip = socket.gethostbyname(socket.gethostname())
    eventData = json.dumps( {"utc":now,"ip":ip,"download":d,"upload":u,"ping":p})
    SendToEventHub(eventData)
    print("sent to event hub")
    if sleepMinutes>0:
        print('sleeping',sleepMinutes)
        time.sleep(sleepMinutes * 10) # time.sleep takes seconds as parameter

ready to start work
GetTimestamp 1688827748 time.struct_time(tm_year=2023, tm_mon=7, tm_mday=8, tm_hour=14, tm_min=49, tm_sec=8, tm_wday=5, tm_yday=189, tm_isdst=0)
{'ip': '98.97.19.239'}
98.97.19.239
17897527.560420774 9300811.476445962 48.418 98.97.19.239
enter SendToEventHub {"utc": 1688827748, "ip": "98.97.19.239", "download": 17897527.560420774, "upload": 9300811.476445962, "ping": 48.418}
Sent data
sent to event hub
sleeping 10
